In [1]:
import pandas as pd
import requests
from io import StringIO

# Corrected URL (space encoded as %20)
url = "https://www.wildfoxindia.com/Kambalghar/itemmastersch_Itemmaster.csv"

try:
    # Add headers in case server blocks Python requests
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Read CSV into pandas DataFrame
    df = pd.read_csv(StringIO(response.text), low_memory=False)

    # Show first 5 rows
    print("✅ CSV loaded successfully. First 5 rows:\n")
    df.head()

except requests.exceptions.RequestException as e:
    print(f"❌ HTTP error while downloading: {e}")

except Exception as e:
    print(f"❌ Error reading CSV into DataFrame: {e}")


✅ CSV loaded successfully. First 5 rows:



In [2]:
df.head()

,articlecode,articlename,barcode,cname1,cname2,cname3,cname4,cname5,cname6,costrate,...,hsn_sac_code,item_name,lev1grpname,lev2grpname,listed_mrp,mrp,partycode,partyname,rate,wsp
0,2386,GIRLS-KIDS WEAR-SLACKS,18123,LITTLE RASCALS,914,16,NaN,NaN,2015,450.0,...,61112000,NaN,GIRLS,KIDS WEAR,745.0,745.0,136.0,"Amrit Apparels.,New Delhi. : SUP013",0.0,745.0
1,2386,GIRLS-KIDS WEAR-SLACKS,901NA20,LITTLE RASCALS,901,20,NaN,NaN,901NA20,365.0,...,61112000,NaN,GIRLS,KIDS WEAR,699.0,699.0,136.0,"Amrit Apparels.,New Delhi. : SUP013",0.0,699.0
2,2386,GIRLS-KIDS WEAR-SLACKS,921NA30,LITTLE RASCALS,921,30,NaN,NaN,921NA30,415.0,...,61112000,NaN,GIRLS,KIDS WEAR,799.0,799.0,136.0,"Amrit Apparels.,New Delhi. : SUP013",0.0,799.0
3,2386,GIRLS-KIDS WEAR-SLACKS,921NA32,LITTLE RASCALS,921,32,NaN,NaN,921NA32,415.0,...,61112000,NaN,GIRLS,KIDS WEAR,799.0,799.0,136.0,"Amrit Apparels.,New Delhi. : SUP013",0.0,799.0
4,2386,GIRLS-KIDS WEAR-SLACKS,927NA30,LITTLE RASCALS,927,30,NaN,NaN,927NA30,535.0,...,61112000,NaN,GIRLS,KIDS WEAR,999.0,999.0,136.0,"Amrit Apparels.,New Delhi. : SUP013",0.0,999.0


In [3]:
df.shape

(587935, 30)

In [ ]:
#itemms,

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587935 entries, 0 to 587934
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   articlecode   587935 non-null  int64  
 1   articlename   587935 non-null  object 
 2   barcode       570829 non-null  object 
 3   cname1        587910 non-null  object 
 4   cname2        586209 non-null  object 
 5   cname3        586039 non-null  object 
 6   cname4        584760 non-null  object 
 7   cname5        536384 non-null  object 
 8   cname6        541153 non-null  object 
 9   costrate      587922 non-null  float64
 10  shrtname      0 non-null       float64
 11  desc1         468046 non-null  object 
 12  desc2         524134 non-null  object 
 13  desc3         1 non-null       object 
 14  desc4         1 non-null       object 
 15  desc5         1 non-null       object 
 16  desc6         0 non-null       float64
 17  ext           587935 non-null  object 
 18  gene

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib
import pyodbc

# ========== CONFIG ==========
DB_SERVER = "SWETHA\\SQLEXPRESS"
DB_DATABASE = "wildfox"
DB_USERNAME = "wildfox"
DB_PASSWORD = "wfx@123"
TABLE_NAME = "itemmaster"

# CSV URL
itemmaster_url = "https://www.wildfoxindia.com/Kambalghar/itemmastersch_Itemmaster.csv"

# ========== LOAD CSV ==========
print("⏳ Loading CSV...")
df = pd.read_csv(itemmaster_url)
print(f"✅ CSV loaded. Shape: {df.shape}")
print(f"📌 Columns: {list(df.columns)}")

# ========== DETECT ODBC DRIVER ==========
drivers = [driver for driver in pyodbc.drivers() if "SQL Server" in driver]
if not drivers:
    raise Exception("❌ No ODBC SQL Server driver found. Please install ODBC Driver 17 or 18 for SQL Server.")
ODBC_DRIVER = drivers[-1]  # use latest installed driver
print(f"✅ Using ODBC Driver: {ODBC_DRIVER}")

# ========== CONNECT TO SQL SERVER ==========
params = urllib.parse.quote_plus(
    f"DRIVER={{{ODBC_DRIVER}}};"
    f"SERVER={DB_SERVER};"
    f"DATABASE={DB_DATABASE};"
    f"UID={DB_USERNAME};"
    f"PWD={DB_PASSWORD};"
    f"TrustServerCertificate=Yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

# ========== CREATE TABLE DYNAMICALLY ==========
def generate_sql_server_table_query(table_name, df):
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        if "int" in str(dtype):
            sql_type = "BIGINT"
        elif "float" in str(dtype):
            sql_type = "DECIMAL(18,4)"
        else:
            sql_type = "NVARCHAR(MAX)"
        col_name = f"[{col}]"
        columns.append(f"{col_name} {sql_type}")
    columns_sql = ",\n    ".join(columns)
    query = f"""
    IF OBJECT_ID('{table_name}', 'U') IS NULL
    CREATE TABLE {table_name} (
        id BIGINT IDENTITY(1,1) PRIMARY KEY,
        {columns_sql}
    );
    """
    return query

create_table_query = generate_sql_server_table_query(TABLE_NAME, df)
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    print(f"✅ Table '{TABLE_NAME}' created (if not exists).")

# ========== INSERT DATA ==========
try:
    df.to_sql(TABLE_NAME, engine, if_exists="append", index=False, chunksize=5000)
    print(f"✅ Inserted {len(df)} rows into '{TABLE_NAME}'.")
except Exception as e:
    print("❌ Error inserting data:", e)
